In [3]:
import edward as ed
from edward.models import Normal, Categorical, Empirical
import tensorflow as tf
import numpy as np

In [2]:
class NNVI():
    
    def __init__(self, D, H, C, sigma_w=1.0, sigma_b=1.0):
        
        self.C = C  # number of unique class
        self.H = H  #hidden_layer size
        self.D = D  # X dimension
        self.X = tf.placeholder(dtype = tf.float32, shape = [None, D] )
        # note that we are multiplying by sigma_w purposely to show what it is.
        self.W_0 = Normal(loc= tf.zeros([D,H]), scale = sigma_w * tf.ones([D,H]))
        self.W_1 = Normal(loc= tf.zeros([H,C]), scale = sigma_w * tf.ones([H,C]))
        self.b_0 = Normal(loc = tf.zeros(H), scale = sigma_b * tf.ones(H))
        self.b_1 = Normal(loc= tf.zeros(C), scale=sigma_b*tf.ones(C))
        #we are adding a vector to a scalar(bias) 
        h = tf.sigmoid(tf.matmul(self.X, self.W_0) + b_0)
        h = tf.matmul(h, W_1) + b_1
        self.Y = Categorical(logits =  (tf.matmul(self.X,self.W)+self.b) )
        self.y_ = tf.placeholder(tf.int32, [None], name = "y_placeholder")
        
    def fit(self, X, Y, epochs, batch_sz):
        # Y should be a vector of class index.
        # ex) np.argmax(mnist.train.labels, axis = 1)
        costs = []
        n_batches = len(X) #batch_sz
        assert self.D == X.shape[1]
        assert self.C == len(set(Y))
        self.T = n_batches * epochs
        print "num epoches: ", epochs, ", num_batch: ", n_batches, ", T: ", self.T
        
        self.qw1 = Normal(loc = tf.Variable(tf.random_normal([self.D,self.C])), 
           scale = tf.nn.softplus(tf.Variable(tf.random_normal([self.D,self.C]))))
        self.qb1 = Normal(loc = tf.Variable(tf.random_normal([self.C])), 
           scale = tf.nn.softplus(tf.Variable(tf.random_normal([self.C]))))
        
        self.sess = tf.InteractiveSession()
        self.inference = ed.KLqp({self.W:self.qw1, self.b:self.qb1}, data={self.Y: self.y_})
        
        self.inference.initialize(n_samples = 5, n_iter = self.T)
        self.sess.run(tf.global_variables_initializer())

        for i in range(epochs):
            # print "epoch: ", i
            from sklearn.utils import shuffle
            X, Y = shuffle(X, Y, random_state=0)
            for j in range(n_batches):
                X_batch = X[j*batch_sz:(j+1)*batch_sz]
                Y_batch = Y[j*batch_sz:(j+1)*batch_sz]
                
                self.info_dict =  self.inference.update(feed_dict= {self.X: X_batch,  self.y_: Y_batch})
                self.inference.print_progress(self.info_dict)
    
    def predict_argmax(self, X, ntest = 10):
        qw1 = self.sess.run(self.qw1.sample(ntest))
        qb1 = self.sess.run(self.qb1.sample(ntest))
        
        preds = []
        for i in xrange(ntest):
            y_post_vi = tf.argmax(tf.matmul(self.X,qw1[i]) +qb1[i] , axis = 1)

            preds.append(y_post_vi)
        preds_result = self.sess.run(preds, feed_dict= {self.X:X})
        return preds_result

In [3]:
import edward as ed
from edward.models import Normal, Categorical, Empirical
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../MNIST_data/", one_hot= True)

def mapping(x):
    map_dict = dict(zip([0,1,2,4,5,6,7,8,9], [0,1,2,3,4,5,6,7,8]))
    return map_dict[x]

X_train = mnist.train.images[np.argmax(mnist.train.labels,axis = 1) !=3]
print "X has a shape of :", X_train.shape
Y_train = np.argmax(mnist.train.labels[np.argmax(mnist.train.labels,axis = 1) !=3], axis = 1)
Y_train = np.array(map(mapping, Y_train))
print "Y has a shape of ", Y_train.shape
print "Y is in the ", set(Y_train)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.


KeyboardInterrupt: 

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
features = ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']
df = pd.read_csv('../winequality-white.csv', sep=';')
X = df[features].values
y = (df.quality >= 7).values


# 标准化处理
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [4]:
X.shape

(4898, 11)

In [ ]:
model_vi = MultipleLogisticRegressionVI(D = 11, C = 1)
model_vi.fit(X= X_train, Y=Y_train, epochs= 100, batch_sz= 1000)

In [8]:
mu = Normal(loc=0.0, scale=1.0)
x = Normal(loc=mu, scale=1.0, sample_shape=10)

qmu = Empirical(tf.Variable(tf.zeros(500)))
inference = ed.HMC({mu: qmu}, {x: np.zeros(10, dtype=np.float32)})

/home/jjh/anaconda3/envs/edward_py2/lib/python2.7/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \


In [9]:
inference.run()

500/500 [100%] ██████████████████████████████ Elapsed: 5s | Acceptance Rate: 0.940


In [12]:
import pandas

ImportError: cannot import name isna